In [3]:
import random, requests
from re import match # allows regex in list


# Avaliador de jogadores de Forca

Começamos esse projeto com uma clase que simula um jogo de forca (com a funcao de advinhar a letra e/ou a palavra):


In [4]:
# -------------- #
# Word pool
url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
r = requests.get(url, allow_redirects=True)
content = str(r.content.decode()).split('\n')
# -------------- #

# We need to remove words like 'Adhemar', because its not lowercase
b=0
for i in range(len(content)):
    if (content[i-b] != content[i-b].lower()):
        content.remove(content[i-b])
        b+=1
    
len(content)

244597

In [5]:
# Class imported from notebook 7 

class Hangman:
    def __init__(self):
        
        if r.status_code==200:
            self.content = content
        else:
            print("Erro: ", r.status_code)
    
    # ----------------------------- #

    def new_game(self, lifes=5):
        self.running = True
        self.lifes = lifes
        self.word = random.choice(self.content)
        return len(self.word)
    
    # ----------------------------- #

    def guess_letter(self, letter):
        if self.lifes > 0:
            if letter in self.word:
                return [idx for idx in range(len(self.word)) if self.word[idx]==letter]
            else:
                self.lifes -= 1
                if self.lifes == 0:
                    self.running = False
                    return False
                else:
                    return []
    
    # ----------------------------- #

    def guess_word(self, word):
        if self.lifes > 0:
            if self.word == word:
                self.running = False
                return True
            else:
                self.lifes = 0
                self.running = False
                return False
            
    # ----------------------------- #

    def check_word(self, word):
        if self.word == word:
            return True
        return False

Para demonstrar o seu funcionamento antes de entrar nos algoritimos:

In [6]:
# Showcase

game = Hangman()
letters = game.new_game()
print("Letters:" + str(letters))

print(game.guess_letter('2'))
print(game.guess_letter('q'))
print(game.guess_letter('a'))
print(game.guess_letter('y'))
print(game.guess_letter('z'))
print(game.guess_letter('x'))

'Lifes: ', game.lifes


Letters:7
[]
[]
[0, 3]
[]
[]
False


('Lifes: ', 0)

---------
## Algoritimo 1: Letra mais provável

O primeiro algoritimo feito simplismente ve qual a probabilidade de cada letra aparecer, considerando todas as palavras ao mesmo tempo e ignorando as suas posições.
Ele não leva em consideração o estado atual do tabuleiro, e sempre chuta letras na mesma ordem. 

In [7]:
# First algorithm: by absolute letter probability, wo word guesses

alphabet = 'abcdefghijklmnopqrstuvwxyz'
probabilities = {letter:0 for letter in alphabet}

# adds 1 for every time that letter appears
for word in content:
    for letter in word:
        probabilities[letter.lower()] += 1

# Orders the probabilities
probabilities = dict(sorted(probabilities.items(), key=lambda item: item[1], reverse=True))


Para testar sua eficácia, simulamos 1000 jogos com ele e vemos a chance de ele ganhar um jogo:

In [8]:
# 1000 runs, checks the win probability

wins = 0
keysProbs = list(probabilities.keys())
for i in range(1000):

    # starts game
    game = Hangman()
    letters = game.new_game()  

    # makes a list to save the guesses
    guessList = []
    for i in range(letters):
        guessList.append('.')

    curLetter = 0 
    while game.running:

        # guesses a letter and saves it 
        letterSpots = game.guess_letter(keysProbs[curLetter])
        if type(letterSpots) != bool:
            for numbs in letterSpots:
                guessList[numbs] = keysProbs[curLetter]
            curLetter+=1

        # checks if the word is already guessed
        guessWord = ''.join(guessList)
        gotWord = game.check_word(guessWord)
        if gotWord:
            break

    if gotWord:
        wins+=1
    # print(guessWord) # uncomment to see what the games ended on!

print('This algorithm wins '+ str(wins/10) +'%. of the time')

This algorithm wins 15.1%. of the time


Como podemos ver, se desconsiderar-mos o estado atual do jogo e sempre chutar em uma mesma ordem, não conseguimos ir muito longe. Ele acaba sendo melhor do que um chute completamente aleatorio, mas não por uma margem muito grande.

---------
## Algoritimo 2: Probabilidade de letra por espaço restante
Esse algoritimo parte do conceito da probabilidade de letra por rodada, mas leva em consideração as letras que estal faltando, e seus lugares. Ele encontra a probabilidade de uma letra ser encontrada em um espaço vazio levando em consideração a lista filtrada.

In [9]:
def letter_by_empty_space(guessList, currentAlphabet):

    # better efficiency (only not guessed letters)
    cutAlphabet = [i for i in currentAlphabet if i not in guessList]  
    probabilities = [{letter:0 for letter in cutAlphabet} for i in range(len(guessList))]

    # gets all the words that fit this situation
    filterRegex = ''.join(guessList)
    possibleWords = list(filter(lambda v: match(filterRegex, v), content))

    # gets words that fit into the current alphabet, and lenght
    filterRegex = '^['+currentAlphabet+']{'+str(len(guessList))+'}$'
    avaliableWords = list(filter(lambda v: match(filterRegex, v), possibleWords))

    # makes a list of probabilities for every spot
    for word in avaliableWords:
        for i in range(len(word)):
            if (word[i] in currentAlphabet) and (word[i] not in guessList):
                probabilities[i][word[i].lower()] += 1

    # just so hit doesnt always guess the first empty
    listIndexes = [i for i in range(len(guessList))]
    random.shuffle(listIndexes)

    # gets the max of an empty spot
    for i in listIndexes:
        if guessList[i] == '.':
            return max(probabilities[i], key=probabilities[i].get)

Para testar sua eficácia, simulamos 300 jogos com ele e vemos a chance de ele ganhar um jogo:

In [35]:
# 300 runs, checks the win probability

wins = 0
troubleWords = []
for i in range(300):

    # starts game
    game = Hangman()
    letters = game.new_game()  

    # makes a list to save the guesses
    guessList = []
    for i in range(letters):
        guessList.append('.')

    nextGuess = 'a'
    currentAlphabet = 'abcdefghijklmnopqrstuvwxyz'
    while game.running:

        # guesses a letter and saves it 
        letterSpots = game.guess_letter(nextGuess)
        
        # saves the guessed (and spots) letter to the list
        if type(letterSpots) != bool:
            for numbs in letterSpots:
                guessList[numbs] = nextGuess

        # removes guessed letter from alphabet if its wrong
        if letterSpots == []:
            currentAlphabet = currentAlphabet.replace(nextGuess,'')

        # checks if the word is already guessed
        guessWord = ''.join(guessList)
        gotWord = game.check_word(guessWord)
        if gotWord:
            break

        # gets the next guess
        nextGuess = letter_by_empty_space(guessList, currentAlphabet)

    if gotWord:
        wins+=1
    else:
        troubleWords.append(game.word)
    # print(guessWord) # uncomment to see what the games ended on!

print('This algorithm wins '+ str(wins/3) +'%. of the time')
print('Trouble words: ' + ', '.join(troubleWords))

This algorithm wins 89.0%. of the time
Trouble words: empunham, emulem, manjariamos, guinada, drenaste, sigilo, festas, suscito, pelemos, decepo, manejaras, assomava, jubilassemos, bufai, fuxicarem, guiaste, filem, folgam, expandem, nevemos, picho, zanzassem, xelim, jorres, jazo, zunirmos, julguei, maximizavam, burles, soou, permuto, digitam, fazes


Esse algoritmo é uma grande melhora comparada ao primeiro algoritimo, mas infelizmente não considera a palavra inteira, e simplesmente chuta em relação a uma posição. Ele falharia com palavras como `soou`, que indica que ele não considera as suas opções muito otimamente. 

---------
## Algoritimo 3: Probabilidade de letra por rodada
A cada rodada, esse algoritimo verifica todas as palavras possíveis partindo do estado atual, utilizando regex. Ele utiliza a palavra atual (chamada guessList), com letras nos locais corretos e '.' em letras que ele ainda não sabe, para criar um filtro na lista de todas as palavras possíveis. Ele então filtra novamente essa lista para remover todos as palavras que não tem as letras que o algoritimo sabe que não esta na palavra.
Ele repete esse processo para todos os seus chutes

In [10]:
# generates a new guess based on the board's state
def get_new_guess(guessList, currentAlphabet):

    probabilities = {letter:0 for letter in currentAlphabet}

    # gets all the words that fit this situation
    filterRegex = ''.join(guessList)
    possibleWords = list(filter(lambda v: match(filterRegex, v), content))

    # gets words that fit into the current alphabet, and lenght
    filterRegex = '^['+currentAlphabet+']{'+str(len(guessList))+'}$'
    avaliableWords = list(filter(lambda v: match(filterRegex, v), possibleWords))

    # adds 1 for every time that letter appears
    for word in avaliableWords:
        letterAdded = []
        for letter in word:
            if (letter in currentAlphabet) and (letter not in guessList) and (letter not in letterAdded):
                probabilities[letter.lower()] += 1
                letterAdded.append(letter)

    # gets the key of the hight value in dict
    nextGuess = max(probabilities, key=probabilities.get)
    return nextGuess

Para testar sua eficácia, simulamos 300 jogos com ele e vemos a chance de ele ganhar um jogo:

In [11]:
# 300 runs, checks the win probability

wins = 0
troubleWords = []
for i in range(300):

    # starts game
    game = Hangman()
    letters = game.new_game()  

    # makes a list to save the guesses
    guessList = []
    for i in range(letters):
        guessList.append('.')

    nextGuess = 'a'
    currentAlphabet = 'abcdefghijklmnopqrstuvwxyz'
    while game.running:

        # guesses a letter and saves it 
        letterSpots = game.guess_letter(nextGuess)
        
        # saves the guessed (and spots) letter to the list
        if type(letterSpots) != bool:
            for numbs in letterSpots:
                guessList[numbs] = nextGuess

        # removes guessed letter from alphabet if its wrong
        if letterSpots == []:
            currentAlphabet = currentAlphabet.replace(nextGuess,'')

        # checks if the word is already guessed
        guessWord = ''.join(guessList)
        gotWord = game.check_word(guessWord)
        if gotWord:
            break

        # gets the next guess
        nextGuess = get_new_guess(guessList, currentAlphabet)

    if gotWord:
        wins+=1
    else:
        troubleWords.append(game.word)
    # print(guessWord) # uncomment to see what the games ended on!

print('This algorithm wins '+ str(wins/3) +'%. of the time')
print('Trouble words: ' + ', '.join(troubleWords))

KeyboardInterrupt: 

Esse algorítmo, principalmente quando comparado com os anteriores, tem uma performance muito boa, com probabilidade de vitória perto de 95%. Ele tambem tem melhora no tempo de processamento em relação a ambos.

Comparado ao algoritimo anterior, que somente analiza uma posição por vez, esse algoritimo analiza todas as letras possiveis restantes na palavra, aumentando sua chance de acerto. Ele tambem so considera uma vez a letra na palavra, então a palavra `arara` só contaria como 1 `a` no total, pois um chute ja revelaria todas.

--------------------

## Media dos resultados dos algoritmos:
- Letra mais provável: 15%
- Probabilidade de letra por espaço restante: 90%
- Probabilidade de letra por rodada: 95%